In [4]:
import pandas as pd
import pyspark
from pyspark.sql.functions import col
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

Importing Amazon reviews through Spark

In [5]:
spark = (pyspark.sql.SparkSession.builder
    .master("local")
    .getOrCreate())

In [3]:
movies_df = spark.read.json('reviews_Movies_and_TV.json')
books_df = spark.read.json('reviews_Books.json')
metadata = spark.read.json('metadata.json')

These are complete datasets from a project by [Julien McAuley.](http://jmcauley.ucsd.edu/data/amazon/links.html) To help narrow my results, I started by scraping the top 100 DC & Marvel Graphic Novels for their ASIN, so that I could start with an understanding of where to find the reviews I want to target. 

In [48]:
#DC Comics - top ID's

from selenium.webdriver import Chrome
browser = Chrome()
url = "https://www.amazon.com/gp/bestsellers/books/4400/ref=pd_zg_hrsr_books"
browser.get(url)
all_ids = [x.get_attribute('href').split('dp/')[1].split('/')[0] 
       for x in browser.find_elements_by_xpath('//*[@id="zg-ordered-list"]/li/span/div/span/a')]


In [51]:
url = "https://www.amazon.com/Best-Sellers-Books-Marvel-Comics-Graphic-Novels/zgbs/books/4400/ref=zg_bs_pg_2?_encoding=UTF8&pg=2"

In [52]:
browser.get(url)
all_ids_2 = [x.get_attribute('href').split('dp/')[1].split('/')[0] 
       for x in browser.find_elements_by_xpath('//*[@id="zg-ordered-list"]/li/span/div/span/a')]


In [53]:
all_ids.extend(all_ids_2)

In [54]:
len(all_ids)

100

In [55]:
all_ids[:100]

['B009OIZ9M8',
 'B00ARKCTES',
 'B00NAHL15I',
 'B00FRP6VKQ',
 'B00AAJQYPG',
 'B076C129TM',
 '1423167228',
 'B07MHVZZZQ',
 'B00NEW45XE',
 'B07BH4HKL9',
 'B01IO3NJVQ',
 'B00DYVESVO',
 'B00AAJR554',
 '0785156593',
 'B00AAJR0GI',
 'B07BH47HB3',
 'B01M6CBVFS',
 'B00ARKCTHA',
 'B074WH8D18',
 'B00AAJR3M4',
 'B00DYVESX2',
 'B07G8M8CS3',
 'B07G8LF78G',
 'B00VU2BKQA',
 'B00NEW45Z2',
 '1302912550',
 'B00K5Y5WG8',
 'B01JT4A2DW',
 '148470438X',
 'B00BC711DK',
 'B071RZ2WPV',
 'B00MC2HKP2',
 '1302915061',
 '1484781422',
 'B076C26GQY',
 '1368026672',
 'B00AAJQX6G',
 'B00EARP0HM',
 'B01IQ44P1Q',
 'B00BC710QI',
 'B00GBDI9O4',
 'B00ARKCTA2',
 '0785164219',
 'B07L43NVCQ',
 'B00J43YD48',
 '130291958X',
 'B00PR4B2S8',
 'B007Y319Y2',
 '0785164030',
 'B072YS7GX7',
 'B075MT477X',
 'B007NACVAW',
 '1302912569',
 'B007I83MPM',
 'B0794BSFJ2',
 '1484732820',
 '0785121056',
 'B07D4QWFS2',
 'B01NH4Y224',
 'B077J89645',
 'B07BH44PC7',
 '1484743318',
 'B00BXQCI32',
 'B07JWH9KJ6',
 '0785157263',
 '0785158685',
 '13029172

In [152]:
import csv

with open('../fantasy_comics.csv', 'w') as f:
    wr = csv.writer(f)
    wr.writerow(all_ids)

In [13]:
books_df.show(3)

+----------+--------+-------+--------------------+-----------+--------------+------------+--------------------+--------------+
|      asin| helpful|overall|          reviewText| reviewTime|    reviewerID|reviewerName|             summary|unixReviewTime|
+----------+--------+-------+--------------------+-----------+--------------+------------+--------------------+--------------+
|0000000116|  [5, 5]|    4.0|Interesting Grish...|04 27, 2002| AH2L9G3DQHHAJ|       chris|  Show me the money!|    1019865600|
|0000000116|  [0, 0]|    1.0|The thumbnail is ...|03 24, 2014|A2IIIDRK3PRRZY|      Helene|Listing is all sc...|    1395619200|
|0000000868|[10, 10]|    4.0|I'll be honest. I...|09 11, 2002|A1TADCM7YWPQ8M|    Joel@AWS|Not a Bad Transla...|    1031702400|
+----------+--------+-------+--------------------+-----------+--------------+------------+--------------------+--------------+
only showing top 3 rows



In [57]:
marvel_meta = metadata.filter(col("asin").isin(all_ids)).toPandas()

In [85]:
marvel_reco = marvel_meta['related'].tolist()
temp = [also[0] for also in marvel_reco]

In [86]:
all_marvel_recos = [item for sublist in temp for item in sublist]

In [88]:
all_marvel_reco_df = metadata.filter(col('asin').isin(all_marvel_recos))

In [90]:
all_marvel_reco_df.count()

426

In [87]:
len(all_marvel_recos)

920

In [77]:
m_recommend = [val for v[0] in marvel_reco for val in v[0]]

NameError: name 'v' is not defined

In [76]:
[item for sublist in l for item in sublist]

['0785123202',
 '0785157050',
 '078512179X',
 '078513297X',
 '0785163182',
 '0785131736',
 '0785164537',
 '0785140794',
 '0785156887',
 '0785156593',
 '0785138838',
 '0785119744',
 '0785125965',
 '0785114823',
 '0785164219',
 '078513882X',
 '0785138811',
 '0785157263',
 '0785161945',
 '0785161953',
 '0785142347',
 '0785120122',
 '0785124373',
 '0785141669',
 '0785166467',
 '0785121781',
 '0785114793',
 '0785142282',
 '0785131728',
 '0785157069',
 '078515650X',
 '0785120521',
 '0785120513',
 '0785117148',
 '078515566X',
 '0785135499',
 '0785118748',
 '078512294X',
 '0785137262',
 '0785137246',
 '0785148108',
 '0785156631',
 '0785124691',
 '0785148558',
 '0785116729',
 '0785137270',
 '0785143416',
 '0785164979',
 '078512750X',
 '0785119841',
 '0785166378',
 '0785123121',
 '0785124462',
 '0785158685',
 '0785143777',
 '0785128204',
 '0785123148',
 '0785164030',
 '0785183795',
 '0785149171',
 '0785183787',
 '0785134506',
 '0785156178',
 '0785121897',
 '078514675X',
 '0785163174',
 '07851550

In [18]:
fantasy_reviews

NameError: name 'fantasy_reviews' is not defined

In [156]:
fantasy_reviews.shape

(8, 9)

In [174]:
metadata.dtypes

[('_corrupt_record', 'string'),
 ('asin', 'string'),
 ('brand', 'string'),
 ('categories', 'array<array<string>>'),
 ('description', 'string'),
 ('imUrl', 'string'),
 ('price', 'double'),
 ('related',
  'struct<also_bought:array<string>,also_viewed:array<string>,bought_together:array<string>,buy_after_viewing:array<string>>'),
 ('salesRank',
  'struct<Appliances:bigint,Arts, Crafts & Sewing:bigint,Automotive:bigint,Baby:bigint,Beauty:bigint,Books:bigint,Camera &amp; Photo:bigint,Cell Phones & Accessories:bigint,Clothing:bigint,Computers & Accessories:bigint,Electronics:bigint,Gift Cards Store:bigint,Grocery & Gourmet Food:bigint,Health & Personal Care:bigint,Home &amp; Kitchen:bigint,Home Improvement:bigint,Industrial & Scientific:bigint,Jewelry:bigint,Kitchen & Dining:bigint,Magazines:bigint,Movies & TV:bigint,Music:bigint,Musical Instruments:bigint,Office Products:bigint,Patio, Lawn & Garden:bigint,Pet Supplies:bigint,Prime Pantry:bigint,Shoes:bigint,Software:bigint,Sports &amp; Outd

In [79]:
more_df = metadata.filter(col('asin').startswith('0785'))

In [28]:
titles = meta_df.select('asin','title').collect()

In [45]:
fant = metadata.filter(col('asin').startswith('B07'))

In [80]:
more_df.count()

6806

In [182]:
meta_df.count()

2077

In [44]:
len(titles)

2077

In [43]:
import csv

with open('all_dc.csv', 'w') as f:
    w = csv.writer(f)
    for val in titles:
        w.writerow((val[0], val[1]))

In [177]:
dc_reviews.to_csv('dc_comic_top100.csv')

In [140]:
dc_reviews.drop(['helpful','reviewText','reviewTime','reviewerName','summary','unixReviewTime'], axis=1, inplace=True)

In [141]:
dc_reviews['asin'].value_counts()

1401232590    307
1401220347    294
1401223176    284
0345391810    178
1401229697     84
1401232051     81
1401229352     72
1401235360     52
1401238963     37
1401242383     31
140123884X     13
1401247113      8
B00CU06XFY      3
Name: asin, dtype: int64

In [36]:
comic_readers = list(set(dc_reviews['reviewerID'].tolist()))

In [32]:
movies_df.show(3)

+----------+-------+-------+--------------------+-----------+--------------+------------------+--------------------+--------------+
|      asin|helpful|overall|          reviewText| reviewTime|    reviewerID|      reviewerName|             summary|unixReviewTime|
+----------+-------+-------+--------------------+-----------+--------------+------------------+--------------------+--------------+
|0000143502| [0, 0]|    5.0|This has some gre...|01 17, 2013|A3R5OBKS7OM2IR|Rebecca L. Johnson| Alton... nough said|    1358380800|
|0000143529| [0, 0]|    5.0|This is a great p...| 10 2, 2013|A3R5OBKS7OM2IR|Rebecca L. Johnson|         Ah Alton...|    1380672000|
|0000143561| [2, 4]|    2.0|I have to admit t...|07 17, 2008| AH3QC2PC1VTGP|   Great Home Cook|Don't waste your ...|    1216252800|
+----------+-------+-------+--------------------+-----------+--------------+------------------+--------------------+--------------+
only showing top 3 rows



In [38]:
movie_reviews = movies_df.filter(col("reviewerID").isin(comic_readers)).toPandas()

In [39]:
movie_reviews.shape

(31343, 9)

In [50]:
movie_reviews.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0005119367,"[0, 0]",5.0,I've watched this movie every six months or so...,"06 11, 2012",AE62TIY3899YL,"Daniel J. Silva ""Superman_242""",Best of the Bible Collection AND best telling ...,1339372800
1,0307141950,"[4, 4]",5.0,I suppose giving this video a 5-star is rather...,"01 24, 2003",A2XVRCU5DQBULH,"L. Varnau ""nerff20""",From the voice of Little Critter...,1043366400
2,0307142493,"[3, 3]",5.0,It is rather ironic to think that the elfin Mi...,"12 3, 2003",A2NJO6YE954DBH,Lawrance M. Bernabo,The charming story of how Kris Kringle became ...,1070409600
3,0307514161,"[1, 1]",5.0,"In just telling the story in the song ""Rudolph...","12 25, 2005",A2NJO6YE954DBH,Lawrance M. Bernabo,The deservedly beloved 1964 Bass-Rakin televis...,1135468800
4,0307514161,"[0, 0]",5.0,I bought a bunch of these old holiday classics...,"12 19, 2010",A3PZ88WU7RPTP6,TeamQuinnHQ,Mine!,1292716800


In [51]:
movie_reviews.drop(['helpful','reviewText','reviewTime','reviewerName','summary','unixReviewTime'], axis=1, inplace=True)

In [55]:
dc_reviews.head()

,asin,overall,reviewerID
0,014038572X,5.0,A2INGRA4M7D0QH
1,014038572X,5.0,AC1MHHZVWJE1P
2,014038572X,5.0,A38Z66H58RIJ64
3,014038572X,1.0,A77UMI1QYXJF9
4,014038572X,5.0,A3BVJTBO17JTZE


In [93]:
def new_id_column(df, column, suffix_val, new_name):
    """Take in column with unique indexes, return new index values"""
    unique_vals = list(set(df[column].tolist()))
    new_ids = [int(str(i) + suffix_val) for i in range(1,len(unique_vals)+1)]
    new_id_dict = {k:v for k,v in zip(unique_vals, new_ids)}
    df[new_name] = df[column].apply(lambda x: new_id_dict[x])

In [77]:
asins = list(set(dc_reviews['asin'].tolist()))
len(asins)

14

In [78]:
dc_id = [int(str(i) + '00') for i in range(1,len(asins)+1)]
len(dc_id)

14

In [79]:
new_id_dict = {k:v for k,v in zip(asins, dc_id)}
dc_reviews['amazon_id'] = dc_reviews['asin'].apply(lambda x: new_id_dict[x])
dc_reviews.head()

,asin,overall,reviewerID,amazon_id
0,014038572X,5.0,A2INGRA4M7D0QH,1300
1,014038572X,5.0,AC1MHHZVWJE1P,1300
2,014038572X,5.0,A38Z66H58RIJ64,1300
3,014038572X,1.0,A77UMI1QYXJF9,1300
4,014038572X,5.0,A3BVJTBO17JTZE,1300


In [80]:
movie_asins = list(set(movie_reviews['asin'].tolist()))
len(movie_asins)

15964

In [81]:
m_id = [int(str(i) + '99') for i in range(1,len(movie_asins)+1)]
len(m_id)

15964

In [83]:
m_id_dict = {k:v for k,v in zip(movie_asins, m_id)}
movie_reviews['amazon_id'] = movie_reviews['asin'].apply(lambda x: m_id_dict[x])
movie_reviews.head()

,asin,overall,reviewerID,amazon_id
0,0005119367,5.0,AE62TIY3899YL,1065299
1,0307141950,5.0,A2XVRCU5DQBULH,1174199
2,0307142493,5.0,A2NJO6YE954DBH,621099
3,0307514161,5.0,A2NJO6YE954DBH,839799
4,0307514161,5.0,A3PZ88WU7RPTP6,839799


In [84]:
all_reviews = pd.concat([dc_reviews, movie_reviews], axis=0)

In [87]:
all_reviews.drop('asin', axis=1, inplace=True)

In [88]:
all_reviews.head()

,overall,reviewerID,amazon_id
0,5.0,A2INGRA4M7D0QH,1300
1,5.0,AC1MHHZVWJE1P,1300
2,5.0,A38Z66H58RIJ64,1300
3,1.0,A77UMI1QYXJF9,1300
4,5.0,A3BVJTBO17JTZE,1300


In [94]:
new_id_column(all_reviews, 'reviewerID', suffix_val='88', new_name='amazon_user_id')

In [96]:
all_reviews.drop('reviewerID', axis=1, inplace=True)

In [169]:
all_reviews.head()

,overall,amazon_id,amazon_user_id
0,5.0,1300,73188
1,5.0,1300,57288
2,5.0,1300,53288
3,1.0,1300,22088
4,5.0,1300,33688


In [107]:
all_reviews_spark = spark.createDataFrame(all_reviews)

In [108]:
all_reviews_spark.persist()

DataFrame[overall: double, amazon_id: bigint, amazon_user_id: bigint]

In [158]:
(train, test) = all_reviews_spark.randomSplit([.8,.2])

In [190]:
# Build the recommendation model using ALS
als = ALS(userCol='amazon_user_id', itemCol='amazon_id', ratingCol='overall', nonnegative=True)

als_model = als.fit(train)

In [191]:
pred = als_model.transform(test)

In [192]:
pred_df = pred.toPandas()

In [193]:
pred_df = pred_df.dropna(axis=0)

In [194]:
pred = spark.createDataFrame(pred_df)

In [195]:
al = als_model.extractParamMap

In [196]:
print(al)

<bound method Params.extractParamMap of ALS_c17a9a3dcb6b>


In [197]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="overall",
                                predictionCol="prediction")

rmse = evaluator.evaluate(pred)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.3328492850220897


In [ ]:
& (pred_df['amazon_id'].astype(str).str.endswith('00'))

In [231]:
pred_df.loc[pred_df['amazon_user_id'] == 94688]

,overall,amazon_id,amazon_user_id,prediction
529,5.0,300,94688,2.584244


In [122]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder


als_model =  ALS(userCol="amazon_user_id", itemCol="amazon_id", ratingCol="overall")

                 
params = ParamGridBuilder().addGrid(als_model.regParam, [0.01,0.001,0.1]).addGrid(als_model.rank, [4,10,50]).build()


## instantiating crossvalidator estimator
cv = CrossValidator(estimator=als_model, estimatorParamMaps=params,evaluator=evaluator,parallelism=4)
best_model = cv.fit(train)    

# We see the best model has a rank of 50, so we will use that in our future models with this dataset
best_model.bestModel.rank

50

In [ ]:
query = """
SELECT 
    asin,
    title
FROM metadta
"""

In [ ]:
metadta.createOrReplaceTempView('metadta')

In [ ]:
met_df = spark.sql(query)

In [ ]:
books = met_df.select("asin", met_df.salesRank['Books']
                     .alias('book_ranks'), "title").filter('book_ranks is not null').collect()

In [ ]:
print(image_comics)

In [ ]:
image_comics = []
for i in range(len(books)):
    if books[i]['book_ranks'] == 4395:
        image_comics.append(books[i])

* non-negative = TRUE
* search latent features
* See the cluster
* Clusters that containt both 
* Combine clusters 
* Reduce ranks for increased 